Experiment with Abishek Thakur's Quora plagiarism detector as an NTN testbed.
[Article on Linkedin](https://www.linkedin.com/pulse/duplicate-quora-question-abhishek-thakur/)

In [1]:
!pip install -q keras==2.2.5
![ -r is_that_a_duplicate_quora_question ] || git clone https://github.com/LanceNorskog/is_that_a_duplicate_quora_question.git
!git pull
%cd is_that_a_duplicate_quora_question

     |████████████████████████████████| 337kB 2.7MB/s 
fatal: not a git repository (or any of the parent directories): .git
/content/is_that_a_duplicate_quora_question


In [2]:
!pip install -q -r requirements.txt

In [3]:
!mkdir -p /tmp/data
%cd /tmp/data
![ -r glove.840B.300d.txt ] || wget -nc http://www-nlp.stanford.edu/data/glove.840B.300d.zip 
![ -r glove.840B.300d.txt ] || unzip glove.840B.300d.zip
![ -r quora_duplicate_questions.tsv ] || wget -nc https://github.com/MLDroid/quora_duplicate_challenge/raw/master/data/quora_duplicate_questions.tsv
![ -r GoogleNews-vectors-negative300.bin.gz ] || wget -nc https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
!python -m nltk.downloader stopwords
!ls
%cd /content/is_that_a_duplicate_quora_question


/tmp/data
Archive:  glove.840B.300d.zip
replace glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
glove.840B.300d.txt  GoogleNews-vectors-negative300.bin.gz
glove.840B.300d.zip  quora_duplicate_questions.tsv
/content/is_that_a_duplicate_quora_question


In [4]:
!pwd
!head /tmp/data/quora_duplicate_questions.tsv
from google.colab import drive
drive.mount('/content/gdrive')


/content/is_that_a_duplicate_quora_question
id	qid1	qid2	question1	question2	is_duplicate
0	1	2	What is the step by step guide to invest in share market in india?	What is the step by step guide to invest in share market?	0
1	3	4	What is the story of Kohinoor (Koh-i-Noor) Diamond?	What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?	0
2	5	6	How can I increase the speed of my internet connection while using a VPN?	How can Internet speed be increased by hacking through DNS?	0
3	7	8	Why am I mentally very lonely? How can I solve it?	Find the remainder when [math]23^{24}[/math] is divided by 24,23?	0
4	9	10	Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?	Which fish would survive in salt water?	0
5	11	12	Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?	I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?	1
6	13	14	Should I buy tiago?	What keeps childern

feature_engineering.py

In [10]:
"""
Detecting duplicate quora questions
feature engineering
@author: Abhishek Thakur
"""

import gc
import pickle as cPickle
import pandas as pd
import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
from nltk import download as nltk_download
nltk_download('punkt')
stop_words = stopwords.words('english')


def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)


def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


def sent2vec(s):
    words = str(s).lower().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

gc.enable()

data = pd.read_csv('/tmp/data/quora_duplicate_questions.tsv', sep='\t')
data = data.drop(['id', 'qid1', 'qid2'], axis=1)

print('a')

data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
#data['diff_len'] = data.len_q1 - data.len_q2
#data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
##data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
##data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
#data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))
#data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
#data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
#data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
#data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
#data['fuzz_partial_token_set_ratio'] = data.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
#data['fuzz_partial_token_sort_ratio'] = data.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
#data['fuzz_token_set_ratio'] = data.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
#data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

print('b')

model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/data/GoogleNews-vectors-negative300.bin.gz', binary=True)
print('b2')
data['wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)
print('gc experiment')
model = None
gc.collect()

print('c')

norm_model = gensim.models.KeyedVectors.load_word2vec_format('/tmp/data/GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model.init_sims(replace=True)
print('c2')
data['norm_wmd'] = data.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)
norm_model = None
gc.collect()

print('d')

question1_vectors = np.zeros((data.shape[0], 300))
error_count = 0

for i, q in tqdm(enumerate(data.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

gc.collect()

question2_vectors  = np.zeros((data.shape[0], 300))
for i, q in tqdm(enumerate(data.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

gc.collect()
print('e')

data['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

#data['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
#                                                          np.nan_to_num(question2_vectors))]

#data['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
#                                                          np.nan_to_num(question2_vectors))]

#data['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
#                                                          np.nan_to_num(question2_vectors))]

#data['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
#                                                          np.nan_to_num(question2_vectors))]

#data['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
#                                                          np.nan_to_num(question2_vectors))]

#data['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
#                                                          np.nan_to_num(question2_vectors))]
print('f')

data['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
#data['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
#data['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

print('g')

cPickle.dump(question1_vectors, open('/tmp/data/q1_w2v.pkl', 'wb'), -1)
cPickle.dump(question2_vectors, open('/tmp/data/q2_w2v.pkl', 'wb'), -1)

print('h')

data.to_csv('/tmp/data/quora_features.csv', index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
a
b
b2


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:28: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


gc experiment
c
c2


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:36: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:37: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


d


0it [00:00, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in double_scalars
404290it [01:56, 3477.35it/s]
404290it [02:00, 3342.03it/s]


e


/usr/local/lib/python2.7/dist-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


f
g
h


In [0]:
data.to_csv('/content/gdrive/My Drive/Colab Notebooks/plagiarism_data/quora_features.csv', index=False)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
